## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,73.78,73,5,19.66,clear sky
1,1,Conde,BR,-7.2597,-34.9075,74.66,94,75,12.66,broken clouds
2,2,Thompson,CA,55.7435,-97.8558,54.05,63,10,2.10,clear sky
3,3,Bredasdorp,ZA,-34.5322,20.0403,57.27,91,54,1.48,broken clouds
4,4,Laguna,US,38.4210,-121.4238,88.43,45,1,5.01,clear sky


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Laguna,US,38.4210,-121.4238,88.43,45,1,5.01,clear sky
18,18,Kloulklubed,PW,7.0419,134.2556,84.70,75,84,9.82,broken clouds
25,25,Sakakah,SA,29.9697,40.2064,75.04,26,0,6.98,clear sky
27,27,San Francisco,US,37.7749,-122.4194,76.98,76,40,4.00,smoke
35,35,Merauke,ID,-8.4667,140.3333,81.88,76,85,9.51,overcast clouds
41,41,Lorengau,PG,-2.0226,147.2712,82.33,79,98,8.41,overcast clouds
48,48,Port Hedland,AU,-20.3167,118.5667,75.25,38,22,9.22,few clouds
54,54,Tucurui,BR,-3.7661,-49.6725,79.47,70,95,4.94,overcast clouds
56,56,Hambantota,LK,6.1241,81.1185,79.02,83,99,28.12,overcast clouds
58,58,Denpasar,ID,-8.6500,115.2167,80.26,78,40,10.36,scattered clouds


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                194
City                   194
Country                194
Lat                    194
Lng                    194
Max Temp               194
Humidity               194
Cloudiness             194
Wind Speed             194
Current Description    194
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head() 

194


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Laguna,US,38.4210,-121.4238,88.43,45,1,5.01,clear sky
18,18,Kloulklubed,PW,7.0419,134.2556,84.70,75,84,9.82,broken clouds
25,25,Sakakah,SA,29.9697,40.2064,75.04,26,0,6.98,clear sky
27,27,San Francisco,US,37.7749,-122.4194,76.98,76,40,4.00,smoke
35,35,Merauke,ID,-8.4667,140.3333,81.88,76,85,9.51,overcast clouds


In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Laguna,US,88.43,clear sky,38.4210,-121.4238,
18,Kloulklubed,PW,84.70,broken clouds,7.0419,134.2556,
25,Sakakah,SA,75.04,clear sky,29.9697,40.2064,
27,San Francisco,US,76.98,smoke,37.7749,-122.4194,
35,Merauke,ID,81.88,overcast clouds,-8.4667,140.3333,
41,Lorengau,PG,82.33,overcast clouds,-2.0226,147.2712,
48,Port Hedland,AU,75.25,few clouds,-20.3167,118.5667,
54,Tucurui,BR,79.47,overcast clouds,-3.7661,-49.6725,
56,Hambantota,LK,79.02,overcast clouds,6.1241,81.1185,
58,Denpasar,ID,80.26,scattered clouds,-8.6500,115.2167,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Laguna,US,88.43,clear sky,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
18,Kloulklubed,PW,84.70,broken clouds,7.0419,134.2556,Storyboard Beach Resort
25,Sakakah,SA,75.04,clear sky,29.9697,40.2064,Fakhamat Al Orjoana Apartment
27,San Francisco,US,76.98,smoke,37.7749,-122.4194,The Clift Royal Sonesta Hotel
35,Merauke,ID,81.88,overcast clouds,-8.4667,140.3333,


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))